## Project Name: Bangladesh Students HSC Result Prediction

### OUTCOMES

- 1. load the dataset

- 2. Y data Profiling (Automatic EDA)
    - Basic Understanding the dataset
    - drop the column
    - Rename or Simply columns
    - Correlation Numberical features VS Target Variable

- 3. Train Test Split
    - Seperate Features Matrix (X) and Target Variable (y)
    - Train Test Split

- 4. Create Pipeline For Our model

    - 1. Pipeline for Numerical Features
    - 2. Pipeline for Categorical Features
    - 3. Combine them (numerical and catagorical Features)

- 5. Model Training and Evaluation
    - Prediction and visualization


- 6. Cross Validation

- 7. Grid Search CV on Our model
    - To Get the best Parameter

- 8. Randomized Search CV
    - To Get the best Parameter

- 9. Comapre Which Parameters is Best for Our model.
    - 9.1: Prediction and Evaluation for Grid search:
    - 9.2: Prediction and Evaluation for Random Search cv:

- 10. Save Model with Best Parameters

- 11. MLFlow Introduce
    - 11.1: Discover MLFlow

    - 11.2: Mlflow with RandomForest model

    - 11.3: Mlflow with our best model

In [ ]:
### === import Necessary Libraries === ###

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# EDA Analysis
from ydata_profiling import ProfileReport

## Sklearn Preprocessing
from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler, OneHotEncoder


from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline


# Model 
from sklearn.linear_model import LinearRegression, Ridge

from sklearn.ensemble import RandomForestRegressor


#### Evaluation Metrices
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV


# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")



### 1. load the dataset

In [3]:
# load
df = pd.read_csv("bangladesh_student_performance.csv")

df.head()

,date,gender,age,address,famsize,Pstatus,M_Edu,F_Edu,M_Job,F_Job,relationship,smoker,tuition_fee,time_friends,ssc_result,hsc_result
0,29/04/2018,M,18,Rural,GT3,Together,3,2,At_home,Farmer,No,No,71672,4,4.22,3.72
1,29/04/2018,F,19,Rural,LE3,Apart,0,4,Other,Health,Yes,No,26085,5,3.47,2.62
2,29/04/2018,F,19,Rural,GT3,Together,0,3,Teacher,Services,No,No,40891,3,3.32,2.56
3,29/04/2018,F,19,Rural,LE3,Apart,2,3,At_home,Business,No,No,50600,2,4.57,4.17
4,29/04/2018,M,17,Rural,GT3,Together,1,1,At_home,Farmer,No,No,62458,2,4.50,3.94


In [4]:
# Shape
df.shape

(2018, 16)

### 2. Y data Profiling (Automatic EDA)
- 2.1: Y data Profiling & Basic Understanding the dataset
- 2.2: drop the column
- 2.3: Rename or Simply columns
- 2.4: Correlation Numberical features VS Target Variable

#### 2.1: Y data Profiling & Basic Understanding the dataset

In [5]:
# y-data profiling

profile = ProfileReport(df, title="HSC Result Prediction ML", explorative = True)

# convert csv file to html 
profile.to_file("ydata_HSC_Result.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 111.38it/s]


-  Basic idea of Student Dataset

- Total feature : 16
- Total Sample : 2018

- famsize:
    - family total members
    - GT3 => Greater than 3 members
    - LF2 => Less than 3 members

- Pstatus:
    - Family Status
    - Member live together or not
    
- M_Edu: 
    - Mother Education Qualification
    - level : 0 => no education
    - level : 1 => jsc or some educated
    - level : 2 => ssc or higger level 1 educated.

- F_Edu:
    - Father Education Qualification
    - level : 0 => no education
    - level : 1 => jsc or some educated
    - level : 2 => ssc or higger level 1 educated.

- M_Job:
    - Mother Job Profile
    - At_home: No job
    - Teacher: Job as a teacher
    - other : others job

- F_Job:
    - Father Job Profile
    - At_home: No job
    - Teacher: Job as a teacher
    - other : others job

- relationship:
    - Student relationship or not
    - Yes: They are relationship 
    - No : They are not relationship.

- tuition_fee:
    - Cost of Tutition free for Student

- time_friends:
    - Time spend with friends
    - count as a level not an hours
    - level : 0 => no spend time with friends
    - level : 1 => some time spend with friends.

- ssc_result:
    - Student Secondary School Certificate Profile
    
- hsc_result:
    - Higher Secondary Certificate Profile
    - This is our target feature

In [5]:
df.head(2)

,date,gender,age,address,famsize,Pstatus,M_Edu,F_Edu,M_Job,F_Job,relationship,smoker,tuition_fee,time_friends,ssc_result,hsc_result
0,29/04/2018,M,18,Rural,GT3,Together,3,2,At_home,Farmer,No,No,71672,4,4.22,3.72
1,29/04/2018,F,19,Rural,LE3,Apart,0,4,Other,Health,Yes,No,26085,5,3.47,2.62


In [6]:
df['tuition_fee'].max()

np.int64(129168)

In [9]:
129168 + 5000

134168

In [7]:
df['M_Edu'].unique()

array([3, 0, 2, 1, 4])

In [8]:
df['time_friends'].unique()

array([4, 5, 3, 2, 1])

- ydata Profiling (Automatical EDA) we get some decision:

- 1. Numerical Features
    - [age, tuition_fee, ssc_result]

- 2. Categorical Features
    - [ gender, address,  famsize, Pstatus, M_Job, F_Job, relationship, smoker, ]

- 3. Target Features
    - [ hsc_result ]

- 4. Numerical but actual Categorical Features
    - [ M_Edu, F_Edu, , time_friends, ]

- 5. Useless Features for ML model
    - [ date ]


- NOTE: 

    - "hsc_result "is highly overall correlated with "ssc_result".
    
    - smoker is highly imbalanced (60.6%).

    - this Problem the useless columns
        - date

        - Therefore, we drop this column

#### 2.2: drop the column

In [9]:
# To see the columns name
df.columns

Index(['date', 'gender', 'age', 'address', 'famsize', 'Pstatus', 'M_Edu',
       'F_Edu', 'M_Job', 'F_Job', 'relationship', 'smoker', 'tuition_fee',
       'time_friends', 'ssc_result', 'hsc_result'],
      dtype='object')

In [10]:
# drop the date column

if 'date' in df.columns:
    df.drop( columns = ['date'], inplace = True)

df.columns.size

15

#### 2.3: Rename or Simply columns
- So we easily understand our features or columns

In [ ]:
# simply the columns apply way_02: - So we easily understand our features or columns

df.columns = [
    'st_gender', 'st_age','st_address', 'st_fam_size', 'pstatus','m_edu', 'f_edu', 'm_job','f_job','st_relationship','smoker', 'tuition_fee', 'time_with_friends','ssc_result','hsc_result'
]

df.head(2)

,st_gender,st_age,st_address,st_fam_size,pstatus,m_edu,f_edu,m_job,f_job,st_relationship,smoker,tuition_fee,time_with_friends,ssc_result,hsc_result
0,M,18,Rural,GT3,Together,3,2,At_home,Farmer,No,No,71672,4,4.22,3.72
1,F,19,Rural,LE3,Apart,0,4,Other,Health,Yes,No,26085,5,3.47,2.62


#### 2.4: Correlation Numberical features VS Target Variable

In [12]:
# Numerical vs Target Correlation

corr_target = df.select_dtypes(include= np.number).corr()['hsc_result'].sort_values(ascending=False)

print("Correlation Between Numerical Features and Target Variable: \n", corr_target)

Correlation Between Numerical Features and Target Variable: 
 hsc_result           1.000000
ssc_result           0.950178
m_edu                0.063776
f_edu                0.054811
tuition_fee          0.038068
st_age              -0.009857
time_with_friends   -0.156356
Name: hsc_result, dtype: float64


### 3. Train Test Split
- 3.1:  Seperate Features Matrix (X) and Target Variable (y)

- 3.2: Train Test Split

#### 3.1:  Seperate Features Matrix (X) and Target Variable (y)

In [13]:
# Features matrix
X = df.drop('hsc_result', axis = 1)

# Target Variable
y = df['hsc_result']

X.shape, y.shape

((2018, 14), (2018,))

In [14]:
# Numerical Features
numeric_features = X.select_dtypes(include= ['int64', 'float64']).columns

# Categorical Features

categorical_features = X.select_dtypes(include=['object']).columns

numeric_features

Index(['st_age', 'm_edu', 'f_edu', 'tuition_fee', 'time_with_friends',
       'ssc_result'],
      dtype='object')

In [15]:
# categorical_features
categorical_features

Index(['st_gender', 'st_address', 'st_fam_size', 'pstatus', 'm_job', 'f_job',
       'st_relationship', 'smoker'],
      dtype='object')

#### 3.2: Train Test Split

In [16]:
# split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

# this is a Regression problem that way we can't use "stratify"

((1614, 14), (404, 14))

### 4. Create Pipeline For Our model

- 4.1. Pipeline for Numerical Features

- 4.2. Pipeline for Categorical Features

- 4.3. Combine them (numerical and catagorical Features)


#### 4.1. Pipeline for Numerical Features
- imputation
- scaler

In [17]:
# Pipeline for numerical features

num_transformer = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='median')),
        ('scalar', StandardScaler())
    ]
)

num_transformer

,steps,"[('imputer', ...), ('scalar', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'median'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


#### 4.2. Pipeline for Categorical Features
- imputation
- OHE

In [18]:
# Pipeline for Categorical features

cat_transformer = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]
)

cat_transformer

,steps,"[('imputer', ...), ('encoder', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'most_frequent'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,categories,'auto'


#### 4.3. Combine them (numerical and catagorical Features)

In [19]:
# combine them
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer,numeric_features),
        
        ('cat', cat_transformer, categorical_features)
    ]
)
preprocessor

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


### 5. Define Model & Model Training and Evaluation
- 5.1: Define our Model
- 5.2: Prediction and visualization

#### 5.1: Define our Model with pipeline

In [21]:
# our model
reg_rf = RandomForestRegressor(n_estimators=100, random_state=42)

In [22]:

# Define our model with pipeline

pipe = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('model', reg_rf)
    ]
)

# train or fit the model
pipe.fit(X_train, y_train)

pipe

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


#### 5.2: Prediction and visualization

In [25]:
# predict
y_pred = pipe.predict(X_test)


#### ==== Evaluation ==== ####
    
# r2 score
r2 = r2_score(y_test, y_pred)
    
# rmse
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
# mean absolute error
mae = mean_absolute_error(y_test, y_pred)



#### ==== append into a dataframe ==== ####
result = []

result.append({
    "Model":"RandomForestRegressor",
    "R2 Score": r2,
    "RMSE": rmse,
    "MAE": mae
}) 

# Result : convert into dataframe
result_df = pd.DataFrame(result).sort_values("R2 Score", ascending= False)   


print(result_df)

                   Model  R2 Score      RMSE       MAE
0  RandomForestRegressor  0.950248  0.136556  0.108201


In [ ]:
# Visualization

plt.figure( figsize = (8,6) )

sns.scatterplot(x=y_test, y= y_pred, alpha = 0.6, color='teal' )

plt.plot( [2,5] , [2,5], color = "red", linestyle = '--'  )

plt.xlabel("Actual HSC Result")
plt.ylabel("Predicted HSC result")

plt.grid(True)
plt.show()

### 6. Cross Validation

In [29]:
# Cross Validation with 5 fold
cv_scores = cross_val_score(pipe,X_train, y_train, scoring = 'neg_mean_squared_error')

cv_rmse = np.sqrt(-cv_scores)

print(cv_rmse)

[0.13825617 0.14202924 0.15369812 0.14773708 0.12940278]


In [36]:
print(f"RMSE: {cv_rmse.mean():.5f}")

print(f"STD: {cv_rmse.std():.5f}")

RMSE: 0.14222
STD: 0.00827


### 7. Grid Search CV on Our model
- To Get the best Parameter

In [37]:
#define the grid
from sklearn.model_selection import GridSearchCV


param_grid = {
    'model__n_estimators' : [100,200] ,
    'model__max_depth': [None,10,20],
    'model__min_samples_split' : [2,5]
}

In [39]:
# Grid Search for our model
grid_searh = GridSearchCV(
    estimator= pipe,
    param_grid= param_grid,
    cv = 5,
    scoring = 'neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

# fit (train) the model
grid_searh.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'model__max_depth': [None, 10, ...], 'model__min_samples_split': [2, 5], 'model__n_estimators': [100, 200]}"
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [51]:
# predict our model for Grid Search

y_pred_grid = grid_searh.predict(X_test)

# r2 score
r2_grid = r2_score(y_test, y_pred_grid)

print("R2 Score (Grid Search): ", r2_grid)

R2 Score (Grid Search):  0.951389298116811


In [44]:
# The best score
print(f"The Best Score (Grid Search): {- grid_searh.best_score_}")

# The best parameter
print(f"\nThe Best Parameters(Grid Search):\n {grid_searh.best_params_}")

The Best Score (Grid Search): 0.14145800733122194

The Best Parameters(Grid Search):
 {'model__max_depth': 10, 'model__min_samples_split': 2, 'model__n_estimators': 200}


### 8. Randomized Search CV
- To Get the best Parameter

In [45]:
# import  
from scipy.stats import randint

# define the parameters of randomized cv

param_dist = {
    'model__n_estimators' : randint(100,250) ,
    'model__max_depth': [None,10,20],
    'model__min_samples_split' : randint(1,10)
}

In [46]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator = pipe,
    param_distributions = param_dist,
    n_iter=1,
    cv = 5 ,
    scoring = 'neg_root_mean_squared_error',
    n_jobs =-1,
    verbose = 2,
    random_state = 42
)

# fit the Random Forest model with random search cv paramaters

random_search.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


,estimator,Pipeline(step...m_state=42))])
,param_distributions,"{'model__max_depth': [None, 10, ...], 'model__min_samples_split': <scipy.stats....001F6E32E7230>, 'model__n_estimators': <scipy.stats....001F6E31F39B0>}"
,n_iter,1
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [49]:
# predict our model for Random Search

y_pred_random = random_search.predict(X_test)

# r2 score
r2_random = r2_score(y_test, y_pred_random)

print("R2 Score (Random Search): ", r2_random)

R2 Score (Random Search):  0.9508976913006011


In [47]:
# The best score
print(f"The Best Score (Random Search): {- random_search.best_score_}")

# The best parameter
print(f"\nThe Best Parameters(Random Search):\n {random_search.best_params_}")

The Best Score (Random Search): 0.14162700978626863

The Best Parameters(Random Search):
 {'model__max_depth': 20, 'model__min_samples_split': 4, 'model__n_estimators': 192}


### 9. Comapre Which Parameters is Best for Our model.

- 9.1: Prediction and Evaluation for Grid search:
    - The Best Score (Grid Search): 0.14145800733122194

    - The Best Parameters(Grid Search):
        - {'model__max_depth': 10, 'model__min_samples_split': 2, 'model__n_estimators': 200}

    - R2 Score (Grid Search):  0.951389298116811
    



- 9.2: Prediction and Evaluation for Random Search cv:

    - The Best Score (Random Search): 0.14162700978626863

    - The Best Parameters(Random Search):
        - {'model__max_depth': 20, 'model__min_samples_split': 4, 'model__n_estimators': 192}

    - R2 Score (Random Search):  0.9508976913006011


- So, the Final Parameters:
    - model__max_depth': 10
    - model__min_samples_split': 2
    - model__n_estimators': 200


In [52]:
# Final model With Best Parameter
best_model_best_params = RandomForestRegressor(
    n_estimators=200,
    max_depth = 10,
    min_samples_split = 2,
    random_state=42
    )

# Define our Final model with pipeline

final_model = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('model', best_model_best_params)
    ]
)

# train or fit the model
final_model.fit(X_train, y_train)

final_model

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [53]:
# Final model Predict

y_pred_final = final_model.predict(X_test)

# r2 score
r2_final = r2_score(y_test, y_pred_final)

print("R2 Score (Fianal Model): ", r2_final)

R2 Score (Fianal Model):  0.951389298116811


### 10. Save Model with Best Parameters

In [55]:
# Save the Final MOdel
import pickle
filename = "final_model.pkl"

with open( filename, "wb" ) as file:
  pickle.dump( final_model, file )

In [56]:
# load the final model 

with open("final_model.pkl" , "rb") as file:
    model_final = pickle.load(file)
    
model_final

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
# 2. The Logic Function
def predict_gpa(gender, age, address, famsize, 
                Pstatus, M_Edu, F_Edu, M_Job, F_Job, 
                relationship, smoker, tuition_fee, time_friends,
                  ssc_result):
    if age is None or age <= 0 or age > 100:
      return "❌ Age must be between 1 and 100."
    
    if ssc_result is None or ssc_result < 2.50 or ssc_result > 5.00:
      return "❌ SSC Result must be between 2.50 and 5.00."
    
    if tuition_fee is None or tuition_fee < 0 or tuition_fee > 134168:
      return "❌ Please enter valid tuition fee (0-134168 BDT)."
    
    # The column names must match your CSV file exactly
    input_df = pd.DataFrame([[
        gender, age, address, famsize, Pstatus, 
        M_Edu, F_Edu, M_Job, F_Job, relationship, 
        smoker, tuition_fee, time_friends, ssc_result

    ]],
      columns=[
        'gender', 'age', 'address', 'famsize', 'Pstatus', 'M_Edu', 'F_Edu', 'M_Job', 'F_Job', 'relationship', 'smoker', 'tuition_fee', 'time_friends', 'ssc_result'
    ])
        # Return formatted result (Clipped 0-5)
    return input_df
    

In [13]:
predict_gpa(gender='M', age=23, address="B", famsize=2, Pstatus="Big", 
        M_Edu="Yes", F_Edu="yes", M_Job="T", F_Job="B", relationship="no", 
        smoker="NO", tuition_fee=20000, time_friends=2, ssc_result= 3.33)

,gender,age,address,famsize,Pstatus,M_Edu,F_Edu,M_Job,F_Job,relationship,smoker,tuition_fee,time_friends,ssc_result
0,M,23,B,2,Big,Yes,yes,T,B,no,NO,20000,2,3.33


### 11. MLFlow Introduce

- 11.1: Discover MLFlow

- 11.2: Mlflow with RandomForest model

- 11.3: Mlflow with our best model

#### 11.1: Discover MLFlow

In [60]:
import mlflow


# Tracking folder
mlflow.set_tracking_uri("mlruns_HSC")

# experiment name
mlflow.set_experiment("HSC Result Predictor")


2026/01/17 21:53:42 INFO mlflow.tracking.fluent: Experiment with name 'HSC Result Predictor' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:e:/Project/hsc-result-prediction-ml/mlruns_HSC/377105159900297807', creation_time=1768665222349, experiment_id='377105159900297807', last_update_time=1768665222349, lifecycle_stage='active', name='HSC Result Predictor', tags={}>

#### 11.2: Mlflow with RandomForest model
- Final Parameters:
    - model__max_depth': 10
    - model__min_samples_split': 2
    - model__n_estimators': 200

In [61]:
# the best parameter of Random Forest
best_params = {
    'n_estimators': 200,
    'max_depth': 10,
    'min_samples_split': 2,
    'random_state': 42
}

#### 11.3: Mlflow with our best model

In [ ]:
# Start Ml flow
import mlflow.sklearn

with mlflow.start_run(run_name= "Fianl_Model"):
    ### === log parameters === ###
    mlflow.log_params(best_params)
    
    ### === log Model name === ###
    mlflow.log_param("model_type", "RandomForestRegressor")
    
    ### === Train the model  === ###
    model_final.fit(X_train,y_train)
   
    
    # Train predict (To check overfit or not)
    y_train_pred = model_final.predict(X_train)
    
    # Train Evaluation
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    
    train_r2 = r2_score(y_train, y_train_pred)
    
    ### === log train details === ###
    mlflow.log_metric("Final model train rmse", train_rmse)
    mlflow.log_metric("Final model train r2", train_r2)
    
    
    
    ### === Test Predict === ###
    # predict
    y_test_pred = model_final.predict(X_test)
    
    # Evaluation
    
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_r2 = r2_score(y_test, y_test_pred)
    
    ### === Log the test detatis === ###
    mlflow.log_metric("Final Model Test rmse", test_rmse)
    mlflow.log_metric("Final Model Test r2", test_r2)


In [69]:
print("Train R2 socre: ", train_r2)
print("Train RMSE socre: ", train_rmse)


print("\nTest R2 socre: ", test_r2)
print("Test RMSE socre: ", test_rmse)


print("\nBest Parameters: ")
best_params

Train R2 socre:  0.9886445870324477
Train RMSE socre:  0.06418416657271542

Test R2 socre:  0.951389298116811
Test RMSE socre:  0.1349799485615145

Best Parameters: 


{'n_estimators': 200,
 'max_depth': 10,
 'min_samples_split': 2,
 'random_state': 42}